In [39]:
#IMPORT AND FUNCTIONS
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import yfinance as yf
import matplotlib.pyplot as plt
import datetime
import math
import numpy as np
pd.set_option('display.max_columns', None)
import tweepy 
import time
import seaborn as sns
import multiindex
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats 
import random
from scipy.optimize import minimize
from IPython.display import display, Markdown, clear_output, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
import ipywidgets as widgets





import warnings
warnings.filterwarnings("ignore")


#IMPORT FUNCTION#############################################################
def import_file_csv(_):
    with out_import_csv:
        clear_output()
        global spy
        global portfolio_df
        global my_ticks
        global df_list
        global  portfolio
        
        user_url = str(folder_path.value) + "/" + str(file_name.value)
        user_url = user_url.replace("\\","/")
        #data received through csv of yahoo finance
        portfolio_path = user_url
        portfolio = pd.read_csv(portfolio_path)
        try:
           my_ticks = list(portfolio.Symbol)
        except:
           my_ticks = list(portfolio.Ticker)
        df_list = [yf.download(tick, period = "2y", interval = "1d") for tick in my_ticks]
        
        try:
           portfolio["Purchase Price"]
        except:
           portfolio["Purchase Price"] = [1]*portfolio.shape[0]
        
        try:
           portfolio["Quantity"]
        except NameError:
           portfolio["Quantity"] = [1]*portfolio.shape[0]
        
        spy = yf.download("SPY", period = "2y", interval = "1d")
        
        #Creation of variables and dataframes
        total_portfolio = 0
        
        #individual new variables
        for i in range(len(my_ticks)):
            df_list[i]["20 day mean"] = df_list[i]["Adj Close"].rolling(20).mean()
            df_list[i]["Upper"] = df_list[i]["20 day mean"] + 2*(df_list[i]["Adj Close"].rolling(20).std())
            df_list[i]["Lower"] = df_list[i]["20 day mean"] - 2*(df_list[i]["Adj Close"].rolling(20).std())
            df_list[i]["Daily Returns"] = (df_list[i]["Adj Close"]/df_list[i]["Adj Close"].shift(1))-1
            df_list[i]["Cumulative Daily Return"] = (1+ df_list[i]["Daily Returns"]).cumprod()
            total_portfolio += df_list[i]["Adj Close"].tail(1)*portfolio.loc[i,"Quantity"]
        
        for i in range(len(my_ticks)):
            df_list[i]["Normed Return"] = df_list[i]["Adj Close"]/df_list[i].iloc[0]["Adj Close"] 
            portfolio.loc[i,"% of allocation"] = float(df_list[i]["Adj Close"].dropna().tail(1))*portfolio.loc[i,"Quantity"]/float(total_portfolio)
        
        #creation of weighted total portfolio over time
        for i in range(len(my_ticks)):
            df_list[i]["Allocation"]= df_list[i]["Normed Return"].dropna()*portfolio.loc[i,"% of allocation"]
            df_list[i]["Position Values"]=df_list[i]["Allocation"].dropna()*float(total_portfolio)
        all_pos = [df["Position Values"].dropna() for df in df_list]
        portfolio_df = pd.concat(all_pos, axis = 1)
        portfolio_df.columns = my_ticks
        portfolio_df = portfolio_df.dropna()
        portfolio_df["Total Position"] = portfolio_df.sum(axis=1)
        portfolio_df["Daily Returns"] = portfolio_df["Total Position"].pct_change(1)
        portfolio_df["Cumulative Daily Return"] = (1+ portfolio_df["Daily Returns"]).cumprod()
        portfolio_df["20 day mean"] = portfolio_df["Total Position"].rolling(20).mean()
        
        #S&P500 reference
        spy["Daily Returns"] = spy["Close"].pct_change(1)
        spy["Cumulative Daily Return"] = (1+ spy["Daily Returns"]).cumprod() 
        
        print("Imported successfully!")
        
def import_file_xls(_):
    with out_import_xls:
        clear_output()
        global spy
        global portfolio_df
        global my_ticks
        global df_list
        global  portfolio
        
        user_url = str(folder_path.value) + "/" + str(file_name.value)
        user_url = user_url.replace("\\","/")
        #data received through csv of yahoo finance
        portfolio_path = user_url
        portfolio = pd.read_xls(portfolio_path)
        try:
           my_ticks = list(portfolio.Symbol)
        except NameError:
           my_ticks = list(portfolio.Ticker)
        
        try:
           portfolio["Purchase Price"]
        except NameError:
           portfolio["Purchase Price"] = [1]*portfolio.shape[0]
        
        try:
           portfolio["Quantity"]
        except NameError:
           portfolio["Quantity"] = [1]*portfolio.shape[0]
        
        df_list = [yf.download(tick, period = "2y", interval = "1d") for tick in my_ticks]
        
        spy = yf.download("SPY", period = "2y", interval = "1d")
        
        #Creation of variables and dataframes
        total_portfolio = 0
        
        #individual new variables
        for i in range(len(my_ticks)):
            df_list[i]["20 day mean"] = df_list[i]["Adj Close"].rolling(20).mean()
            df_list[i]["Upper"] = df_list[i]["20 day mean"] + 2*(df_list[i]["Adj Close"].rolling(20).std())
            df_list[i]["Lower"] = df_list[i]["20 day mean"] - 2*(df_list[i]["Adj Close"].rolling(20).std())
            df_list[i]["Daily Returns"] = (df_list[i]["Adj Close"]/df_list[i]["Adj Close"].shift(1))-1
            df_list[i]["Cumulative Daily Return"] = (1+ df_list[i]["Daily Returns"]).cumprod()
            total_portfolio += df_list[i]["Adj Close"].tail(1)*portfolio.loc[i,"Quantity"]
        
        for i in range(len(my_ticks)):
            df_list[i]["Normed Return"] = df_list[i]["Adj Close"]/df_list[i].iloc[0]["Adj Close"] 
            portfolio.loc[i,"% of allocation"] = float(df_list[i]["Adj Close"].dropna().tail(1))*portfolio.loc[i,"Quantity"]/float(total_portfolio)
        
        #creation of weighted total portfolio over time
        for i in range(len(my_ticks)):
            df_list[i]["Allocation"]= df_list[i]["Normed Return"].dropna()*portfolio.loc[i,"% of allocation"]
            df_list[i]["Position Values"]=df_list[i]["Allocation"].dropna()*float(total_portfolio)
        all_pos = [df["Position Values"].dropna() for df in df_list]
        portfolio_df = pd.concat(all_pos, axis = 1)
        portfolio_df.columns = my_ticks
        portfolio_df = portfolio_df.dropna()
        portfolio_df["Total Position"] = portfolio_df.sum(axis=1)
        portfolio_df["Daily Returns"] = portfolio_df["Total Position"].pct_change(1)
        portfolio_df["Cumulative Daily Return"] = (1+ portfolio_df["Daily Returns"]).cumprod()
        portfolio_df["20 day mean"] = portfolio_df["Total Position"].rolling(20).mean()
        
        #S&P500 reference
        spy["Daily Returns"] = spy["Close"].pct_change(1)
        spy["Cumulative Daily Return"] = (1+ spy["Daily Returns"]).cumprod() 
        
        print("Imported successfully!")
        

    
#ANALYSIS PER STOCK##########################################################
def stock_analysis(_):
    with out_stock:
        clear_output()
        for i in range(len(my_ticks)):
            tick = my_ticks[i]
            print("\n\n\n\n"+'\033[1m'+'\033[94m' +tick+'\033[0m')
            
            purch = portfolio.loc[i,"Purchase Price"]
            last_price =  df_list[i].tail(1)["Adj Close"]
            share = portfolio.loc[i,"Quantity"]
            gain= float(last_price-purch)
            mean_return = df_list[i]["Daily Returns"].dropna().mean()*100
            annualized_return = ((((mean_return/100)+1)**365)-1)*100
            mean_return_last20days = df_list[i]["Daily Returns"].dropna().tail(20).mean()*100
            annualized_return_last20days = ((((mean_return_last20days/100)+1)**365)-1)*100
            
            beta, alpha, r_value, p_value, std_err = stats.linregress(df_list[i]["Daily Returns"].dropna().tail(400), spy["Daily Returns"].dropna().tail(400))
            
            if gain >= 0:
                print('\033[1m' +"Gain : "'\033[0m',end= " ") 
                print('\033[32m',"+",round(float(gain*share)),"$",end= "   ")
                print("+",round(float(gain/purch)*100),"%",'\033[0m')
            else:
                print('\033[1m' +"Loss : "'\033[0m',end= " ") 
                print('\033[31m',round(float(gain*share)),"$",end= "   ")
                print(round(float(gain/purch)*100),"%",'\033[0m')
            
            fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,5))
            fig.suptitle(tick)
            ax1.grid()
            ax1.plot(df_list[i].index, df_list[i]["Adj Close"])
            ax1.set_title("Last 2 years")
            
        
            ax2.grid()
            ax2.plot(df_list[i].tail(50).index, df_list[i]["Adj Close"].tail(50), alpha = 1)
            ax2.plot(df_list[i].tail(50).index, df_list[i][["20 day mean", "Upper", "Lower"]].tail(50), alpha = (.4), linestyle = "--")
            if gain >= 0:
                ax2.axhline(y=purch, color='g', linestyle='dotted', alpha=.5)
            else:
                ax2.axhline(y=purch, color='r', linestyle="-.", alpha = .5)
            
            ax2.set_title("Last 20 days")
            fig.tight_layout()
            plt.show()
            
            print('\033[1m' + "\nBeta: "'\033[0m', round(beta, 5), end= "   ")
            print('\033[1m' + "Alpha: "'\033[0m', round(alpha,5))
            
            fig, (ax21, ax22) = plt.subplots(1, 2,figsize=(20,5))
            ax21.grid()
            ax21.bar(x= df_list[i].resample(rule="w").mean().index, height=df_list[i]["Volume"].resample(rule="w").mean(), width = 4)
            ax21.axhline(y=df_list[i]["Volume"].resample(rule="w").mean().mean(), color='black', linestyle='dotted', alpha=.5)
            ax21.set_ylim(0,df_list[i]["Volume"].resample(rule="w").mean().max()*1.1)
            
            
            ax22.grid()
            ax22 = df_list[i]["Cumulative Daily Return"].plot()
            ax22 = spy["Cumulative Daily Return"].plot()
            ax22.legend([tick, "S&P500"])
            ax22.set_title("Cumulative returns last 2 years")
            fig.tight_layout()
            plt.show()
        
            print('\033[1m' +"Mean Daily Return: "'\033[0m',
                  round(mean_return,2),"%",) 
            print('\033[1m' +"Annualized Return: "'\033[0m',
                  round(annualized_return,2),"%") 
            print('\033[1m' +"\nMean Daily Return (last 20 days): "'\033[0m',
                  round(mean_return_last20days,2),"%",) 
            print('\033[1m' +"Annualized Return (last 20 days): "'\033[0m',
              round(annualized_return_last20days,2),"%") 
            plt.figure(figsize=(10,5))
            plt.hist(df_list[i]["Daily Returns"].dropna(), bins = 100)
            plt.axvline(x= mean_return,color='black', linestyle="-", alpha=.5 )
            plt.title("Daily Returns last 2 years")
            plt.xlim(-0.4,+0.4)
            plt.show()
            
            
#ANALYSIS PORTFOLIO##########################################################
def portfolio_analysis(_):
    with out_portfolio:
        clear_output()
        print("\n\n\n\n"+'\033[1m'+'\033[94m' +"PORTFOLIO"+'\033[0m')
        
        mean_return = portfolio_df["Daily Returns"].dropna().mean()*100
        annualized_return = ((((mean_return/100)+1)**365)-1)*100
        mean_return_last20days = portfolio_df["Daily Returns"].dropna().tail(20).mean()*100
        annualized_return_last20days = ((((mean_return_last20days/100)+1)**365)-1)*100
        beta, alpha, r_value, p_value, std_err = stats.linregress(portfolio_df["Daily Returns"].dropna().tail(400), spy["Daily Returns"].dropna().tail(400))
        
        fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,5))
        fig.suptitle("Portfolio")
        ax1.grid()
        ax1.plot(portfolio_df["Total Position"].index, portfolio_df["Total Position"])
        ax1.set_title("Last 2 years")
        
        ax2.grid()
        ax2.plot(portfolio_df["Total Position"].tail(50).index, portfolio_df["Total Position"].tail(50), alpha = 1)
        ax2.plot(portfolio_df["20 day mean"].tail(50).index, portfolio_df["20 day mean"].tail(50), alpha = (.4), linestyle = "--")
        
        ax2.set_title("Last 20 days")
        fig.tight_layout()
        plt.show()
        
        print('\033[1m' + "\nBeta: "'\033[0m', round(beta, 5), end= "   ")
        print('\033[1m' + "Alpha: "'\033[0m', round(alpha,5))
        
        fig, (ax21, ax22) = plt.subplots(1, 2,figsize=(20,5))
        print('\033[1m' +"Mean Daily Return: "'\033[0m',
              round(mean_return,2),"%",) 
        print('\033[1m' +"Annualized Return: "'\033[0m',
              round(annualized_return,2),"%") 
        print('\033[1m' +"\nMean Daily Return (last 20 days): "'\033[0m',
              round(mean_return_last20days,2),"%",) 
        print('\033[1m' +"Annualized Return (last 20 days): "'\033[0m',
              round(annualized_return_last20days,2),"%") 
        ax21.hist(portfolio_df["Daily Returns"].dropna(), bins = 100)
        ax21.axvline(x= mean_return,color='black', linestyle="-", alpha=.5 )
        ax21.set_title("Daily Returns last 2 years")
        ax21.set_xlim(-0.4,+0.4)
        
        ax22.grid()
        ax22 = portfolio_df["Cumulative Daily Return"].plot()
        ax22 = spy["Cumulative Daily Return"].plot()
        ax22.legend(["Total Portfolio", "S&P500"])
        ax22.set_title("Cumulative returns last 2 years")
        fig.tight_layout()
        plt.show()
        
        plt.show()
        
        portfolio_df[my_ticks].plot(figsize=(24.5,10), alpha=0.8)
        plt.plot(portfolio_df["Total Position"])
        plt.show()
        
        #CORRELATION BETWEEN STOCKS
        fig, ax = plt.subplots(figsize=(10,10))  
        corr_matrix = portfolio_df[my_ticks].join(spy["Adj Close"], how="left").corr()
        sns.heatmap(corr_matrix,annot= True)
        new_labels= my_ticks.copy()
        new_labels.append("S&P500")
        print('\033[1m' +"Average Correlation : "'\033[0m', round(corr_matrix[my_ticks].mean().mean(),3) )
        print('\033[1m' +"Average Correlation with S&P500: "'\033[0m', round(corr_matrix["Adj Close"].mean(),3) )
        ax.set_xticklabels(new_labels)
        ax.set_yticklabels(new_labels)
        plt.show()
            
        
        

#SHARPE RATIO OPTIMIZATION###################################################
def optimization(_):
    with out_optimization:
        
        clear_output()
        stocks = portfolio_df[my_ticks]
        log_ret= np.log(stocks/stocks.shift(1))
        
        num_ports = 10000
        all_weights = np.zeros((num_ports, len(stocks.columns)))
        ret_arr = np.zeros(num_ports)
        vol_arr = np.zeros(num_ports)
        sharpe_arr = np.zeros(num_ports)
        
        for ind in range(num_ports):
            weights = np.array(np.random.random(len(stocks.columns)))
            weights = weights/np.sum(weights)
            all_weights[ind,:] = weights
            ret_arr[ind]=  np.sum((log_ret.mean()*weights)*252)
            vol_arr[ind] = np.sqrt(np.dot(weights.T,np.dot(log_ret.cov()*252,weights)))
            sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]
        
        
        def get_ret_vol_sr(weights):
            """
            Takes in weights, returns array or return,volatility, sharpe ratio
            """
            weights = np.array(weights)
            ret = np.sum(log_ret.mean() * weights) * 252
            vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
            sr = ret/vol
            return np.array([ret,vol,sr])
        
        def neg_sharpe(weights):
            return  get_ret_vol_sr(weights)[2] * -1
        
        # Contraints
        def check_sum(weights):
            '''
            Returns 0 if sum of weights is 1.0
            '''
            return np.sum(weights) - 1
        
        cons = ({'type':'eq','fun': check_sum})
        # 0-1 bounds for each weight
        bounds = ((0, 1),)*len(stocks.columns)
        init_guess = list(np.repeat(1/len(stocks.columns), len(stocks.columns)))
        opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
        
        # Create a linspace number of points to calculate x on
        frontier_y = np.linspace(0.1,0.9,20) 
        def minimize_volatility(weights):
            return  get_ret_vol_sr(weights)[1] 
        frontier_volatility = []
        
        for possible_return in frontier_y:
            # function for return
            cons = ({'type':'eq','fun': check_sum},
                    {'type':'eq','fun': lambda w: get_ret_vol_sr(w)[0] - possible_return})
            
            result = minimize(minimize_volatility,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
            
            frontier_volatility.append(result['fun'])
        plt.figure(figsize=(12,8))
        plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
        plt.colorbar(label='Sharpe Ratio')
        plt.xlabel('Volatility')
        plt.ylabel('Return')
        
        # Add frontier line
        plt.plot(frontier_volatility,frontier_y,'g--',linewidth=3)
        plt.show()
        
        opt_results.x
        opt_values = get_ret_vol_sr(opt_results.x)
        actual_values = get_ret_vol_sr(portfolio["% of allocation"])
        
        print('\033[1m' +"Actual Values \n"'\033[0m')
        print('\033[1m' +"Return :"'\033[0m',actual_values[0])
        print('\033[1m' +"Volatility :"'\033[0m',actual_values[1])
        print('\033[1m' +"Sharpe Ratio :"'\033[0m',actual_values[2])
        
        print('\033[1m' +"\n\nOptimization results \n"'\033[0m')
        print('\033[1m' +"Return :"'\033[0m',opt_values[0])
        print('\033[1m' +"Volatility :"'\033[0m',opt_values[1])
        print('\033[1m' +"Sharpe Ratio :"'\033[0m',opt_values[2])
        
        for i in range(len(my_ticks)):
            print("\n",'\033[1m'+my_ticks[i]+'\033[0m',"\nActual: ",round(portfolio.loc[i,"% of allocation"],2),"     Optimal: ",round(opt_results.x[i],2))

            
            
#SHARPE RATIO OPTIMIZATION (custom)##########################################
def optimization_custom(_):
    
    with out_optimization_custom:
        
        clear_output()

        opt_or_vol = radiobuttons1.value
        days = int_slider2.value
        max_volatility = int_slider1.value/100
        
        
        
        print(f"Sharpe Ratio optimization, last {days} days")
        stocks = portfolio_df[my_ticks].tail(days)
        log_ret= np.log(stocks/stocks.shift(1))
        
        num_ports = 10000
        all_weights = np.zeros((num_ports, len(stocks.columns)))
        ret_arr = np.zeros(num_ports)
        vol_arr = np.zeros(num_ports)
        sharpe_arr = np.zeros(num_ports)
        
        for ind in range(num_ports):
            weights = np.array(np.random.random(len(stocks.columns)))
            weights = weights/np.sum(weights)
            all_weights[ind,:] = weights
            ret_arr[ind]=  np.sum((log_ret.mean()*weights)*252)
            vol_arr[ind] = np.sqrt(np.dot(weights.T,np.dot(log_ret.cov()*252,weights)))
            sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]
        
        
        def get_ret_vol_sr(weights):
            """
            Takes in weights, returns array or return,volatility, sharpe ratio
            """
            weights = np.array(weights)
            ret = np.sum(log_ret.mean() * weights) * 252
            vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
            sr = ret/vol
            return np.array([ret,vol,sr])
        
        def neg_sharpe(weights):
            return  get_ret_vol_sr(weights)[2] * -1
        
        # Contraints
        def check_sum(weights):
            '''
            Returns 0 if sum of weights is 1.0
            '''
            return np.sum(weights) - 1
        
        cons = ({'type':'eq','fun': check_sum})
        # 0-1 bounds for each weight
        bounds = ((0, 1),)*len(stocks.columns)
        init_guess = list(np.repeat(1/len(stocks.columns), len(stocks.columns)))
        opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
        
        # Create a linspace number of points to calculate x on
        frontier_y = np.linspace(0.1,0.9,20) 
        def minimize_volatility(weights):
            return  get_ret_vol_sr(weights)[1] 
        frontier_volatility = []
        
        for possible_return in frontier_y:
            # function for return
            cons = ({'type':'eq','fun': check_sum},
                    {'type':'eq','fun': lambda w: get_ret_vol_sr(w)[0] - possible_return})
            
            result = minimize(minimize_volatility,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
            
            frontier_volatility.append(result['fun'])
        plt.figure(figsize=(12,8))
        plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
        plt.colorbar(label='Sharpe Ratio')
        plt.xlabel('Volatility')
        plt.ylabel('Return')
        
        # Add frontier line
        plt.plot(frontier_volatility,frontier_y,'g--',linewidth=3)
        plt.show()
        
        opt_results.x
        opt_values = get_ret_vol_sr(opt_results.x)
        actual_values = get_ret_vol_sr(portfolio["% of allocation"])
        if opt_or_vol == "optimal":
            print('\033[1m' +"Actual Values \n"'\033[0m')
            print('\033[1m' +"Return :"'\033[0m',actual_values[0])
            print('\033[1m' +"Volatility :"'\033[0m',actual_values[1])
            print('\033[1m' +"Sharpe Ratio :"'\033[0m',actual_values[2])
            
            print('\033[1m' +"\n\nOptimization results \n"'\033[0m')
            print('\033[1m' +"Return :"'\033[0m',opt_values[0])
            print('\033[1m' +"Volatility :"'\033[0m',opt_values[1])
            print('\033[1m' +"Sharpe Ratio :"'\033[0m',opt_values[2])
            
            for i in range(len(my_ticks)):
                print("\n",'\033[1m'+my_ticks[i]+'\033[0m',"\nActual: ",round(portfolio.loc[i,"% of allocation"],2),"     Optimal: ",round(opt_results.x[i],2))
        elif opt_or_vol == "volatility":
                
            cons = ({'type':'eq','fun': check_sum},
                    {'type':'eq','fun': lambda w: get_ret_vol_sr(w)[1] - max_volatility})
            vol_result = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
            vol_values = get_ret_vol_sr(vol_result.x)
            if actual_values[1] > max_volatility:
                print('\033[91m'+"Impossible to reach desired volatility with given stocks\n\n"+'\033[0m')
            print('\033[1m' +"Actual Values \n"'\033[0m')
            print('\033[1m' +"Return :"'\033[0m',actual_values[0])
            print('\033[1m' +"Volatility :"'\033[0m',actual_values[1])
            print('\033[1m' +"Sharpe Ratio :"'\033[0m',actual_values[2])
            
            print('\033[1m' +"\n\nOptimization results \n"'\033[0m')
            print('\033[1m' +"Return :"'\033[0m',vol_values[0])
            print('\033[1m' +"Volatility :"'\033[0m',vol_values[1])
            print('\033[1m' +"Sharpe Ratio :"'\033[0m',vol_values[2])
            
            for i in range(len(my_ticks)):
                print("\n",'\033[1m'+my_ticks[i]+'\033[0m',"\nActual: ",round(portfolio.loc[i,"% of allocation"],2),"     Optimal: ",round(vol_result.x[i],2))
            
        else:
            print("error")


############################################################################
folder_path = widgets.Text(
       value="C:/Users/jacop/Downloads",
       description='Insert folder path', )
file_name = widgets.Text(
       value='quotes.csv',
       description='Insert file name', )
button_import_csv = widgets.Button(description='Import CSV')
out_import_csv = widgets.Output()
button_import_xls = widgets.Button(description='Import XLS')
out_import_xls = widgets.Output()
button_stock = widgets.Button(description='Run  Stocks Analysis')
out_stock = widgets.Output()
button_portfolio = widgets.Button(description='Run Portfolio analysis')
out_portfolio = widgets.Output()
button_optimization = widgets.Button(description='Run Optimization')
out_optimization = widgets.Output()
button_optimization_custom = widgets.Button(description='Run Custom Optimization')
out_optimization_custom = widgets.Output()


button_import_xls.on_click(import_file_xls)
button_import_csv.on_click(import_file_csv)
button_stock.on_click(stock_analysis)
button_portfolio.on_click(portfolio_analysis)
button_optimization.on_click(optimization)


#Variables for custom optimization
radiobuttons1 = widgets.RadioButtons(
    value='optimal', 
    options=['optimal', 'volatility'], 
    description='Parameters')
int_slider1 = widgets.IntSlider(
    value=30, 
    min=0, max=50, step=1, 
    description='Max volatility %')
int_slider2 = widgets.IntSlider(
    value=200, 
    min=0, max=500, step=1, 
    description='days')

button_optimization_custom.on_click(optimization_custom)




l = widgets.Layout(flex='0 1 auto', height='600px', min_height='10px', width='auto', margin="30px")
box_import = widgets.VBox([folder_path, file_name, button_import_csv,button_import_xls,out_import_csv,out_import_xls] )
box_stock = widgets.VBox([button_stock, out_stock])
box_portfolio = widgets.VBox([button_portfolio, out_portfolio ])
box_optimization = widgets.VBox([button_optimization, out_optimization])
box_optimization_custom = widgets.VBox([radiobuttons1,int_slider1, int_slider2, button_optimization_custom, out_optimization_custom])

# defining a list with the contents of our windows
children = [box_import, box_stock,box_portfolio,box_optimization, box_optimization_custom]
# initializing a tab
tab = widgets.Tab()
# setting the tab windows 
tab.children = children
# changing the title of the first and second window
tab.set_title(0, 'Import File')
tab.set_title(1, 'Stock Analysis')
tab.set_title(2, 'Portfolio Analysis')
tab.set_title(3, 'Optimization')
tab.set_title(4, 'Optimization Custom')

tab
#voila "C:/Users/jacop/Portfolio Analysis interactive-Copy1.ipynb"